# Swalokal - Machine Learning

## Import Libraries

In [2]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [14]:
import os
from os import getcwd

import tensorflow_hub as hub
import tensorflow as tf

from tqdm import tqdm

import shutil

print("\u2022 Using TensorFlow Version:", tf.__version__)
print('\u2022 GPU Device Found.' if tf.test.is_gpu_available() else '\u2022 GPU Device Not Found. Running on CPU')

• Using TensorFlow Version: 2.12.0
• GPU Device Found.


In [7]:
print(tf.__version__)

2.12.0


## Import MobileNetV2 as our Transfered Model

In [8]:
module_selection = ("mobilenet_v2", 224, 1280)  # @param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(
    handle_base
)
IMAGE_SIZE = (pixels, pixels)
print(
    "Using {} with input size {} and output dimension {}".format(
        MODULE_HANDLE, IMAGE_SIZE, FV_SIZE
    )
)

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


You can change the model with several list at the first-commented line, but we decided to use mobilenet_v2 for this project.

## Import Dataset from Gdrive

In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [9]:
path = os.path.join(os.getcwd(), "gdrive", "My Drive")
datapath = os.path.join(path, "Capstone")
train_path = os.path.join(datapath, "train")
test_path = os.path.join(datapath, "test")

In [10]:
train_path

'/content/gdrive/My Drive/Capstone/train'

## Data Augmentation for train dataset

In [11]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 20
IMG_SIZE = (200, 200)

train_datagen= ImageDataGenerator(rescale=1./255.,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale = 1./255.)

train_dataset = train_datagen.flow_from_directory(
    train_path, shuffle=True, batch_size=BATCH_SIZE, target_size=IMG_SIZE
)
test_dataset = test_datagen.flow_from_directory(
    test_path, shuffle=True, batch_size=BATCH_SIZE, target_size=IMG_SIZE
)

Found 461 images belonging to 3 classes.
Found 56 images belonging to 3 classes.


## Modelling

### Feature extracting

In [12]:
do_fine_tuning = False  # @param {type:"boolean"}

In [15]:
feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape=IMAGE_SIZE + (3,),
    output_shape=[FV_SIZE],
    trainable=do_fine_tuning,
)

In [16]:
model = tf.keras.Sequential(
    [feature_extractor, tf.keras.layers.Dense(3, activation="softmax")]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 3)                 3843      
                                                                 
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


### (Optional) Unfreeze some layers

In [17]:
NUM_LAYERS = 30  # @param {type:"slider", min:1, max:50, step:1}

if do_fine_tuning:
    feature_extractor.trainable = True

    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable = True

else:
    feature_extractor.trainable = False

### Model fitting

In [18]:
if do_fine_tuning:
    model.compile(
        optimizer=tf.keras.optimizers.SGD(lr=0.002, momentum=0.9),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"],
    )
else:
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

In [19]:
EPOCHS = 15

hist = model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


24/24 [==============================] - 111s 4s/step - loss: 0.9531 - accuracy: 0.5293 - val_loss: 0.5291 - val_accuracy: 0.8036
Epoch 2/15
24/24 [==============================] - 10s 435ms/step - loss: 0.5088 - accuracy: 0.8134 - val_loss: 0.3378 - val_accuracy: 0.9107
Epoch 3/15
24/24 [==============================] - 10s 433ms/step - loss: 0.3842 - accuracy: 0.8720 - val_loss: 0.2762 - val_accuracy: 0.9107
Epoch 4/15
24/24 [==============================] - 10s 432ms/step - loss: 0.3098 - accuracy: 0.8980 - val_loss: 0.2452 - val_accuracy: 0.9107
Epoch 5/15
24/24 [==============================] - 11s 443ms/step - loss: 0.2539 - accuracy: 0.9197 - val_loss: 0.1844 - val_accuracy: 0.9286
Epoch 6/15
24/24 [==============================] - 10s 438ms/step - loss: 0.2463 - accuracy: 0.9262 - val_loss: 0.1815 - val_accuracy: 0.9464
Epoch 7/15
24/24 [==============================] - 9s 393ms/step - loss: 0.2146 - accuracy: 0.9479 - val_loss: 0.1613 - val_accuracy: 0.9464
Epoch 8/15
24

## Save and Export the Model

In [20]:
SWALOKAL_SAVED_MODEL = "swalokal_saved_model"

In [21]:
tf.saved_model.save(model, SWALOKAL_SAVED_MODEL)

In [22]:
%%bash -s $SWALOKAL_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 3)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


2023-05-29 03:43:00.283288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [23]:
loaded = tf.saved_model.load(SWALOKAL_SAVED_MODEL)

In [24]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'keras_layer_input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_layer_input')})
{'dense': TensorSpec(shape=(None, 3), dtype=tf.float32, name='dense')}


## Convert The Model using TFLite's Converter

### Save model to tflite

In [25]:
converter = tf.lite.TFLiteConverter.from_saved_model(SWALOKAL_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [26]:
tflite_model_file = "converted_model.tflite"

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

### Save model into .pb 

In [27]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output from tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get output data
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.12275767 0.11284676 0.76439553]]


### Zip the whole model

In [28]:
shutil.make_archive("swalokal_saved_model", "zip", "swalokal_saved_model")

'/content/swalokal_saved_model.zip'